In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import cv2

import maxflow

from VidFrame_utils import *
from visualization_utils import *
from video_utils import * 

from VidFrame import *

from tqdm.notebook import tqdm

from skimage.color import rgb2gray
from skimage.filters import sobel
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float, img_as_ubyte
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

%load_ext autoreload
%autoreload 2

### Image Pre-Processing (Optional)

In [ ]:
H = 1080 // 3
W = 1920 // 3

vid_frames = load_video('videos/1_Basketball.mp4', H, W)
print(vid_frames.shape)

In [ ]:
n_clusters = 500
compactness = 20

In [ ]:
fig = segment_frames(vid_frames, [0,100,200,300], 2, 2, n_clusters, compactness)
plt.show()

In [ ]:
vid_labels = preprocess_video(vid_frames[:100], tqdm, n_clusters, compactness)

In [ ]:
np.save('preprocess_cache/1_Basketball.npy', vid_labels)

In [ ]:
# vidwrite_from_numpy('outputs/1_Basketball.mp4', vid_frames)

### Visualizing Segmented Frames

In [ ]:
H = 1080 // 3
W = 1920 // 3

vid_frames = load_video('videos/1_Basketball.mp4', H, W)[:100]
label_frames = np.load('preprocess_cache/1_Basketball.npy')

In [ ]:
F, H, W, C = vid_frames.shape

segmented_vid_frames = np.zeros_like(vid_frames)

for f in tqdm(range(F)):
    segmented_vid_frames[f] = get_boundary_segments(vid_frames[f], label_frames[f])

# vidwrite_from_numpy('outputs/1_Baketball_Segmented.mp4', segmented_vid_frames)

In [ ]:
plt.figure()

fig = visualize_images(segmented_vid_frames[80:84], 2, 2)

plt.show()

In [ ]:
VidFrame80 = VidFrame(vid_frames[80], label_frames[80])
VidFrame81 = VidFrame(vid_frames[81], label_frames[81])

In [ ]:
jaccard_threshold = 0.4

mpixels = VidFrame81.calculateMotionSuperpixels(VidFrame80, jaccard_threshold)

print(mpixels)

In [ ]:
imgs = np.zeros((3, H, W, 3), dtype=label_frames[80].dtype)

imgs[0] = segmented_vid_frames[80]
imgs[1] = segmented_vid_frames[81]

motion_superpix_img = np.copy(segmented_vid_frames[81])

for label in mpixels:
    label_pix_x, label_pix_y = np.where(label_frames[81] == label) 
        
    motion_superpix_img[label_pix_x,label_pix_y,:] = np.array([0,255,0])

imgs[2] = motion_superpix_img

fig = visualize_images(imgs, 1, 3)
plt.show()

In [ ]:
N_background_segments = 25
background_compactness = 10

In [ ]:
superpix_mask, background_mask = get_superpixel_masks(vid_frames[81], 
                                                      label_frames[81],
                                                      mpixels)

background_labels = slic_segment_image(
    vid_frames[81], n_segments=N_background_segments, 
    compactness=background_compactness, mask=background_mask
)

background_boundary = get_boundary_segments(vid_frames[81],
                                            background_labels)

plt.figure()
plt.imshow(label_frames[81])
plt.show()

plt.figure()
plt.imshow(background_labels)
plt.show()

N_background_segments = np.unique(background_labels)

plt.figure()
plt.imshow(background_boundary)
plt.show()

n_objs, obj_labels = get_num_foreground_regions(superpix_mask)

plt.figure()
plt.imshow(obj_labels)
plt.show()

### Fit Gaussian Mixture Models for Background and Foreground

In [ ]:
b_mogs, f_mogs = generate_gaussians(vid_frames[81], background_labels, 
                                    obj_labels, n_components=3)

N_b, N_f = len(b_mogs), len(f_mogs)

In [ ]:
t_k = np.zeros(N_f)

sizes_b, priors_b = calculate_background_priors(background_labels)
sizes_f, priors_f = calculate_foreground_priors(obj_labels, t_k, N_f)

# print(priors_b)
# print(priors_f)

unary_background = calculate_unary_potential(vid_frames[81], label_frames[81], priors_b, b_mogs)
unary_foreground = calculate_unary_potential(vid_frames[81], label_frames[81], priors_f, f_mogs)

# print(unary_background)
# print(unary_foreground)

In [ ]:
n_superpixels = label_frames[81].max() + 1

graph = maxflow.Graph[float]()

nodeids = graph.add_grid_nodes(n_superpixels)

# graph.add_grid_edges(nodeids, 1)

graph.add_grid_tedges(nodeids, unary_foreground, unary_background)

In [ ]:
mflow = graph.maxflow()
sgm = graph.get_grid_segments(nodeids)

print(mflow)

In [ ]:
obj_mask = graph_segmentation_to_object_mask(H, W, label_frames[81], sgm)

plt.figure()
plt.imshow(obj_mask, cmap='gray')

In [ ]:
blank_img = np.zeros((H, W))

white_lines = mark_boundaries(blank_img, label_frames[81], color=(1,1,1))[:,:,0]

plt.figure()
plt.imshow(white_lines, cmap='gray')

w_x, w_y = np.where(white_lines == 1)

In [ ]:
n_superpixels = label_frames[81].max() + 1

graph = maxflow.Graph[float]()

nodeids = graph.add_grid_nodes(n_superpixels)

# graph.add_grid_edges(nodeids, 1)

graph.add_grid_tedges(nodeids, unary_foreground, unary_background)

create_adjacency_edges(vid_frames[81], label_frames[81], graph, f_mogs, b_mogs, 500)

In [ ]:
mflow = graph.maxflow()

print(mflow)

sgm = graph.get_grid_segments(nodeids)

obj_mask = graph_segmentation_to_object_mask(H, W, label_frames[81], sgm)

plt.figure()
plt.imshow(obj_mask, cmap='gray')